In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

<h1>1. Подготовка данных</h1>

In [67]:
df = pd.read_csv('all_letters_info.csv')
label_to_letter = df[['label', 'letter']].drop_duplicates().set_index('label')['letter'].to_dict()
letter_to_label = df[['label', 'letter']].drop_duplicates().set_index('letter')['label'].to_dict()
print(label_to_letter)
print(letter_to_label)
df.head()

{1: 'а', 2: 'б', 3: 'в', 4: 'г', 5: 'д', 6: 'е', 7: 'ё', 8: 'ж', 9: 'з', 10: 'и', 11: 'й', 12: 'к', 13: 'л', 14: 'м', 15: 'н', 16: 'о', 17: 'п', 18: 'р', 19: 'с', 20: 'т', 21: 'у', 22: 'ф', 23: 'х', 24: 'ц', 25: 'ч', 26: 'ш', 27: 'щ', 28: 'ъ', 29: 'ы', 30: 'ь', 31: 'э', 32: 'ю', 33: 'я'}
{'а': 1, 'б': 2, 'в': 3, 'г': 4, 'д': 5, 'е': 6, 'ё': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33}


,letter,label,file,background
0,а,1,01_01.png,0
1,а,1,01_02.png,0
2,а,1,01_03.png,0
3,а,1,01_04.png,0
4,а,1,01_05.png,0


In [3]:
images_folder_path = 'all_letters_image/all_letters_image/'

X, Y = [], []

for _, row in df.iterrows():
    image_path = os.path.join(images_folder_path, row['file'])
    
    with Image.open(image_path) as img:
        img = img.convert('L')
        
        img = img.resize((32, 32))
        
        img_flattened = np.array(img).flatten()
        
        X.append(img_flattened)
        Y.append(row['label'])

X = np.array(X)
y = np.array(Y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

<h1>2. Random Forest Classifier</h1>

In [56]:
def test_model(model, X_train, X, y_train, y, NN = False, only_train = False):
    def test_case(X_to_predict, Y_to_predict):
        predictions = model.predict(X_to_predict)
        #[print(prediction) for prediction in predictions]
        if NN: predictions = np.argmax(predictions, axis=1)
        print(predictions)
        #print(len(set(predictions)))
        incorrect_indices = np.where(predictions != Y_to_predict)[0]
        for i in incorrect_indices:
            print(f"Prediction: {label_to_letter[predictions[i]]}, Actual letter: {label_to_letter[Y_to_predict[i]]}")
        
        print(f"Accuracy: {accuracy_score(Y_to_predict, predictions)}, My Accuracy: {(len(X_to_predict) - len(incorrect_indices))/len(X_to_predict)}")


    print("Training data test")
    test_case(X_train, y_train)
    
    if not(only_train):
        print("Full data test")
        test_case(X, y)

In [5]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
test_model(clf, X_train, X, y_train, y)

Training data test
33
[]
Accuracy: 1.0, My Accuracy: 1.0
Full data test
33
[    8    14    27    31    35    41    47    63    99   101   102   113
   135   167   168   169   173   177   181   184   217   218   247   280
   284   286   290   299   304   311   315   327   353   357   360   379
   385   410   411   431   465   468   473   476   487   511   533   538
   542   568   586   592   608   624   683   696   708   720   732   735
   736   747   749   772   794   795   811   833   836   862   868   872
   900   902   903   907   914   927   937   970   977   993  1010  1022
  1026  1034  1039  1053  1061  1078  1083  1087  1107  1111  1114  1121
  1139  1175  1197  1199  1213  1225  1247  1253  1261  1270  1310  1320
  1350  1351  1358  1368  1370  1372  1373  1393  1398  1418  1433  1453
  1454  1475  1482  1483  1488  1489  1493  1516  1530  1533  1534  1547
  1562  1575  1579  1580  1586  1593  1598  1599  1606  1611  1617  1623
  1624  1634  1655  1659  1662  1683  1688  1697 

<h1>3. Многослойный перцептрон (с полносвязными слоями)</h1>

In [6]:
print(X_train)
X_train.shape
#X_train = X_train / 255.0

[[171 169 164 ... 170 171 164]
 [155 157 151 ... 161 167 166]
 [119 111 124 ... 145 131 128]
 ...
 [177 177 177 ... 171 171 170]
 [115 114 111 ... 116 115 113]
 [114 113 113 ... 107 105 105]]


(12771, 1024)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
X_train_normalized = X_train / 255.0
X_normalized = X / 255.0

#Мб услоднить модель?
fully_connected_model = Sequential([
    Dense(1024, input_shape=(1024,), activation='relu'),
    BatchNormalization(),
    #Dropout(0.25),
    Dense(512, activation='relu'),
    BatchNormalization(),
    #Dropout(0.25),
    Dense(256, activation='relu'),
    BatchNormalization(),
    #Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    #Dropout(0.1),
    Dense(34, activation='softmax')
])

fully_connected_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
history = fully_connected_model.fit(X_train_normalized, y_train, epochs=50, batch_size=64, validation_split=0.2)
test_model(fully_connected_model, X_train_normalized, X_normalized, y_train, y, NN=True)

Epoch 1/50


160/160 [==============================] - 6s 22ms/step - loss: 3.1597 - accuracy: 0.1412 - val_loss: 3.7880 - val_accuracy: 0.0513
Epoch 2/50
160/160 [==============================] - 3s 20ms/step - loss: 2.7932 - accuracy: 0.2106 - val_loss: 8.8352 - val_accuracy: 0.0262
Epoch 3/50
160/160 [==============================] - 3s 21ms/step - loss: 2.6289 - accuracy: 0.2511 - val_loss: 10.2985 - val_accuracy: 0.0270
Epoch 4/50
160/160 [==============================] - 3s 20ms/step - loss: 2.5093 - accuracy: 0.2863 - val_loss: 8.5908 - val_accuracy: 0.0305
Epoch 5/50
160/160 [==============================] - 3s 20ms/step - loss: 2.3707 - accuracy: 0.3179 - val_loss: 18.0489 - val_accuracy: 0.0254
Epoch 6/50
160/160 [==============================] - 3s 20ms/step - loss: 2.2918 - accuracy: 0.3319 - val_loss: 17.6704 - val_accuracy: 0.0297
Epoch 7/50
160/160 [==============================] - 3s 20ms/step - loss: 2.1250 - accuracy: 0.3775 - val_loss: 7.7806 - val_accuracy: 0

<h1>4. Сверточная нейронная сеть на исходных данных (с разными вариантами архитектуры).</h1>

In [9]:
X_train_reshaped = X_train_normalized.reshape((-1, 32, 32, 1))

<h3>Simple CNN</h3>

In [10]:
simple_CNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(34, activation='softmax')
])

In [11]:
simple_CNN.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

simple_CNN_history = simple_CNN.fit(X_train_reshaped, y_train, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
160/160 [==============================] - 3s 16ms/step - loss: 3.5135 - accuracy: 0.0356 - val_loss: 3.4873 - val_accuracy: 0.0564
Epoch 2/10
160/160 [==============================] - 2s 15ms/step - loss: 3.4328 - accuracy: 0.0542 - val_loss: 3.3895 - val_accuracy: 0.0481
Epoch 3/10
160/160 [==============================] - 2s 15ms/step - loss: 3.3627 - accuracy: 0.0680 - val_loss: 3.3596 - val_accuracy: 0.0759
Epoch 4/10
160/160 [==============================] - 2s 14ms/step - loss: 3.3204 - accuracy: 0.0930 - val_loss: 3.2955 - val_accuracy: 0.0896
Epoch 5/10
160/160 [==============================] - 2s 14ms/step - loss: 3.2257 - accuracy: 0.1165 - val_loss: 3.1777 - val_accuracy: 0.1084
Epoch 6/10
160/160 [==============================] - 2s 15ms/step - loss: 3.0714 - accuracy: 0.1451 - val_loss: 3.0244 - val_accuracy: 0.1491
Epoch 7/10
160/160 [==============================] - 2s 14ms/step - loss: 2.9350 - accuracy: 0.1734 - val_loss: 2.9076 - val_accuracy: 0.180

<h3>Как мы можем видеть, простая CNN не в состонии развить лучшую стратегию классификации, чем распознавать каждое изображение, как одну и ту же букву. Конечно, можно было бы попытаться настроить гиперпараметры, количество эпох в целях улучшеиня точности классификации, но рациональнее и целесообразнее производить тонкую настройку гиперпараметров на изначально более сложной архитектуре.</h3>

<h3>Более сложная CNN</h3>

In [16]:
more_complex_CNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    #BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(40, (3, 3), activation='relu'),
    #BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(50, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(34, activation='softmax')
])

#X_train_reshaped

In [17]:
more_complex_CNN.compile(optimizer=Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

more_complex_CNN_history = more_complex_CNN.fit(X_train_reshaped, y_train, epochs=10, batch_size=128, validation_split=0.2)
test_model(more_complex_CNN, X_train_reshaped, X, y_train, y, NN=True, only_train=True)

Epoch 1/10
80/80 [==============================] - 4s 34ms/step - loss: 3.5205 - accuracy: 0.0304 - val_loss: 3.5096 - val_accuracy: 0.0290
Epoch 2/10
80/80 [==============================] - 3s 32ms/step - loss: 3.4643 - accuracy: 0.0416 - val_loss: 3.4084 - val_accuracy: 0.0521
Epoch 3/10
80/80 [==============================] - 2s 31ms/step - loss: 3.3835 - accuracy: 0.0658 - val_loss: 3.3619 - val_accuracy: 0.0806
Epoch 4/10
80/80 [==============================] - 2s 30ms/step - loss: 3.2272 - accuracy: 0.1088 - val_loss: 3.0340 - val_accuracy: 0.1366
Epoch 5/10
80/80 [==============================] - 2s 30ms/step - loss: 2.8115 - accuracy: 0.2051 - val_loss: 2.6898 - val_accuracy: 0.2200
Epoch 6/10
80/80 [==============================] - 2s 30ms/step - loss: 2.5722 - accuracy: 0.2668 - val_loss: 2.5470 - val_accuracy: 0.2689
Epoch 7/10
80/80 [==============================] - 2s 30ms/step - loss: 2.4318 - accuracy: 0.3102 - val_loss: 2.4342 - val_accuracy: 0.3104
Epoch 8/10
80

<h3>Как мы видим, добавление большего количества слоев и нейронов на каждом слое улучшило точность классификации, добавив больше эпох мы бы получили достойную(по метрике accuracy) модель</h3>

<h3>Модель с наиболее сложной архитектурой(арбитрарно)</h3>

In [76]:
complex_CNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    Flatten(),
    #Dropout(0.5),
    Dense(128, activation='relu'),
    #Dropout(0.5),
    Dense(34, activation='softmax')
])

In [90]:
complex_CNN.compile(optimizer=Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

complex_CNN_history = complex_CNN.fit(X_train_reshaped, y_train, epochs=10, batch_size=128, validation_split=0.2)
test_model(complex_CNN, X_train_reshaped, X, y_train, y, NN=True, only_train=True)

Epoch 1/10
80/80 [==============================] - 11s 119ms/step - loss: 2.1049 - accuracy: 0.4253 - val_loss: 0.8561 - val_accuracy: 0.7546
Epoch 2/10
80/80 [==============================] - 11s 137ms/step - loss: 0.5232 - accuracy: 0.8437 - val_loss: 0.6802 - val_accuracy: 0.8004
Epoch 3/10
80/80 [==============================] - 11s 133ms/step - loss: 0.3722 - accuracy: 0.8897 - val_loss: 0.6498 - val_accuracy: 0.8149
Epoch 4/10
80/80 [==============================] - 13s 160ms/step - loss: 0.2999 - accuracy: 0.9129 - val_loss: 0.6259 - val_accuracy: 0.8215
Epoch 5/10
80/80 [==============================] - 14s 180ms/step - loss: 0.2728 - accuracy: 0.9225 - val_loss: 0.6135 - val_accuracy: 0.8223
Epoch 6/10
80/80 [==============================] - 11s 133ms/step - loss: 0.2472 - accuracy: 0.9284 - val_loss: 0.6268 - val_accuracy: 0.8153
Epoch 7/10
80/80 [==============================] - 10s 122ms/step - loss: 0.2452 - accuracy: 0.9282 - val_loss: 0.6132 - val_accuracy: 0.8290

<h3>После создания сверточной нейронной сети и настройки гиперпараметров удалось получить модель, корректно классифицирующую русские буквы с вероятностью 94%(на тестовой выборке)</h3>

<h1>4. Сверточная нейронная сеть на расширенных данных.</h1>

<h3>Я не очень понимаю, что требуется сделать в этом пункте, так как если я обучу нейросеть на всех данных, то на чем мне ее тестировать, поэтому я сейчас сделаю скрипт и буду сам рисовать </h3>

In [102]:
images_folder_path = 'additional_data'

X_additional, y_additional = [], []

for root, dirs, files in os.walk(images_folder_path):
    for file in files:
            file_path = os.path.join(root, file)
            
            with Image.open(file_path) as img:
                img = img.convert('L')  # Convert to grayscale
                #img = img.resize((32, 32))  # Resize the image
                
                img_flattened = np.array(img).flatten()
                
                # Print the directory name
                directory_name = os.path.basename(root)
                
                X_additional.append(img_flattened)
                y_additional.append(letter_to_label[directory_name])


In [103]:
y_additional = np.array(y_additional)

X_additional = np.array(X_additional) 

X_additional_reshaped = X_additional.reshape((-1, 32, 32, 1))
print(len(X_additional_reshaped))
# complex_CNN.compile(optimizer=Adam(learning_rate=0.001),
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])

# complex_CNN_history = complex_CNN.fit(X_additional_reshaped, y_additional, epochs=10, batch_size=64, validation_split=0.1)
test_model(complex_CNN, X_additional_reshaped, X, y_additional, y, NN=True, only_train=True)
#test_model(complex_CNN, X_additional_reshaped, X, y_additional, y, NN=True, only_train=True)

print(X_train_reshaped[0])
print(X_additional_reshaped[0])


1651
Training data test
52/52 [==============================] - 1s 10ms/step
[23 29 28 ... 23 28 21]
Prediction: х, Actual letter: ё
Prediction: ы, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: х, Actual letter: ё
Prediction: х, Actual letter: ё
Prediction: в, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: в, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ю, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: х, Actual letter: ё
Prediction: х, Actual letter: ё
Prediction: ф, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: ъ, Actual letter: ё
Prediction: щ, Actual letter: а
Prediction: х, Actual letter: а
Prediction: с, Actual letter: а
Prediction: т, Actual letter: а
Prediction: х, Actual letter: а
Prediction: ы, Actual letter: а
Prediction: ц, Actual letter: а
Prediction: щ, Actual letter: а
Pr

In [96]:
test_model(complex_CNN, X_train_reshaped, X, y_train, y, NN=True, only_train=True)

Training data test
400/400 [==============================] - 3s 8ms/step
[25 16 10 ...  5  6 32]
Prediction: и, Actual letter: н
Prediction: в, Actual letter: ь
Prediction: л, Actual letter: п
Prediction: ж, Actual letter: ш
Prediction: ж, Actual letter: к
Prediction: н, Actual letter: к
Prediction: г, Actual letter: е
Prediction: ё, Actual letter: б
Prediction: н, Actual letter: ж
Prediction: х, Actual letter: я
Prediction: е, Actual letter: ь
Prediction: ы, Actual letter: и
Prediction: щ, Actual letter: ш
Prediction: п, Actual letter: а
Prediction: е, Actual letter: с
Prediction: ч, Actual letter: г
Prediction: ы, Actual letter: а
Prediction: ж, Actual letter: т
Prediction: н, Actual letter: ы
Prediction: д, Actual letter: у
Prediction: е, Actual letter: ё
Prediction: к, Actual letter: н
Prediction: э, Actual letter: ф
Prediction: г, Actual letter: ю
Prediction: щ, Actual letter: а
Prediction: з, Actual letter: ъ
Prediction: з, Actual letter: ф
Prediction: э, Actual letter: я
Predic

In [98]:
print(X_train_reshaped[0])
print(X_additional_reshaped[0])

[[[0.4627451 ]
  [0.47058824]
  [0.47058824]
  ...
  [0.4627451 ]
  [0.4627451 ]
  [0.47058824]]

 [[0.46666667]
  [0.47058824]
  [0.47058824]
  ...
  [0.46666667]
  [0.4627451 ]
  [0.46666667]]

 [[0.4627451 ]
  [0.47058824]
  [0.46666667]
  ...
  [0.47058824]
  [0.46666667]
  [0.47058824]]

 ...

 [[0.50980392]
  [0.51372549]
  [0.50980392]
  ...
  [0.50980392]
  [0.50588235]
  [0.50980392]]

 [[0.50588235]
  [0.50980392]
  [0.51372549]
  ...
  [0.50980392]
  [0.51372549]
  [0.50980392]]

 [[0.51372549]
  [0.50980392]
  [0.50980392]
  ...
  [0.50980392]
  [0.51372549]
  [0.50980392]]]
[[[6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]
  ...
  [6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]]

 [[6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]
  ...
  [6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]]

 [[6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]
  ...
  [6.03086294e-08]
  [6.03086294e-08]
  [6.03086294e-08]]

 ...

 [[6.03086294e-08]
  [6.03086294e-